In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from imp import reload
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import IPython.display as ipd

from tensorflow.keras import datasets, layers, models

import textwrap
import random
import json

from simpletransformers.classification import ClassificationModel
import pandas as pd

In [3]:
import os

In [ ]:
data_path = '../noncomm_use_subset/pmc_json/'

In [ ]:
# Load up files from a directory

In [25]:
dir_path = '../noncomm_use_subset/pmc_json_test/'
filename = 'PMC1616946.xml.json'

print(dir_path + filename)


for file in os.listdir(dir_path):
    filepath = dir_path + filename
    print(filepath)
    with open(filepath, 'r') as infile:
        json_object = json.load(infile)
        print(json.dumps(json_object, indent=2))
    
            #df = pd.read_json(infile,
            #                 lines=True,
            #                 orient='columns')
    #df.head()


../noncomm_use_subset/pmc_json_test/PMC1616946.xml.json
../noncomm_use_subset/pmc_json_test/PMC1616946.xml.json
{
  "paper_id": "PMC1616946",
  "metadata": {
    "title": "Antisense-induced ribosomal frameshifting",
    "authors": [
      {
        "first": "Clark",
        "middle": [
          "M."
        ],
        "last": "Henderson",
        "suffix": "",
        "email": null,
        "affiliation": {}
      },
      {
        "first": "Christine",
        "middle": [
          "B."
        ],
        "last": "Anderson",
        "suffix": "",
        "email": null,
        "affiliation": {}
      },
      {
        "first": "Michael",
        "middle": [
          "T."
        ],
        "last": "Howard",
        "suffix": "",
        "email": null,
        "affiliation": {}
      }
    ]
  },
  "body_text": [
    {
      "text": "The standard triplet readout of the genetic code can be reprogrammed by signals in the mRNA to induce ribosomal frameshifting [reviewed in (1\u20133)]

# Create a class object to represent papers/articles with contained methods

In [30]:
stopwords = []

def normalize(word):
    
    out_word = word.lower()
    
    return out_word

#######################################################################
# Class method for representing Section
#######################################################################

# Section class encapsulates loading, displaying, and simple helper operations 
# e.g. return text or bag of words representation
class Section:
    # Initialize with key article fields from JSON object
    # body_text (should preserve section type)
    def __init__(self, json_input):#, label_map):
        
        # Extract
        self.type=json_object['section']
        self.text=json_object['text']
        self.normalized_text=[normalize(word) for word in self.text.split()]
        self.bow = set(self.normalized_text.split())
        
    def get_text(self):
        return self.text
        
        
# Encapsulates list of sections, takes abstract and body_text JSON list 
# and generates subsections in order
class Sections:
    # Initialize with key article fields from JSON object
    # body_text (should preserve section type)
    def __init__(self, abstract=[], body_text=[]):#, label_map):
        
        # Create a section from the abstract
        
        # Iterate through the elements of the body_text and create sections, 
        # combining sequential items of the same type
        
        # Extract
        self.type=json_object['section']
        self.text=json_object['text']
        self.normalized_text=[]
        

#######################################################################
# Class method for representing Article (list of Sections)
#######################################################################

# Article class encapsulates loading, displaying, and simple helper operations 
# e.g. return text or bag of words representation
class Article:
    # Initialize with key article fields from JSON representation
    # paper_id, title, authors, abstract, body_text, sections
    def __init__(self, json_input):#, label_map):
        
        # Extract
        self.paperID=json_object['paper_id'] #Paper ID
        self.abstract=''
        self.body_text=''

    # 
    def get_text(self):
        return self.text + ' ' + self.body_text
    
    
    def get_bow(self, stopwords=[]):
        return self.card + ' ' + self.answer
    
    
    # Add methods for running ML

In [29]:
"asdfSDFSDF".lower()

'asdfsdfsdf'